In [1]:
from datasets import load_dataset

dataset = load_dataset('thangquang09/fake-new-imposter-hunt-in-texts')

/home/thangquang09/CODE/CTAI_MachineLearning/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating case2_validation split: 100%|██████████| 100/100 [00:00<00:00, 29096.80 examples/s]


In [2]:
dataset

DatasetDict({
    case1_train: Dataset({
        features: ['file_1', 'file_2', 'label'],
        num_rows: 150
    })
    case1_validation: Dataset({
        features: ['file_1', 'file_2', 'label'],
        num_rows: 20
    })
    case2_train: Dataset({
        features: ['file_1', 'file_2', 'label'],
        num_rows: 756
    })
    case2_validation: Dataset({
        features: ['file_1', 'file_2', 'label'],
        num_rows: 100
    })
})